In [1]:
table = "multipolygons"
db = 'dgt_osmpt'

dbset = "local"

notkeys = [
    "ogc_fid", "osm_id", "osm_way_id", "name",
    "type", "admin_level", "boundary",
    'other_tags', 'wkb_geometry', 'lulc_cls'
]

otbl = '/home/jasp/mystuff/multitags_cls.xlsx'

In [3]:
import numpy as np
import pandas as pd
from glass.prop.sql import cols_name
from glass.sql.q import q_to_obj
from glass.wt import obj_to_tbl
from glass.wt.sql import df_to_db

In [4]:
# Get table columns

cols = cols_name(db, table, dbset=dbset)
cols = [c.replace('"', '') for c in cols if c not in notkeys]
cols.sort()

In [5]:
colsql = ", ".join([f"tbl.{c}" for c in cols])
q = f"SELECT {colsql} FROM {table} AS tbl GROUP BY {colsql}"

df = q_to_obj(db, q, dbset=dbset)

In [6]:
# Replace None for empty string

for c in df.columns.values:
    df[c] = np.where(df[c].isnull(), '', c + '=>' + df[c])

In [7]:
# Get column with all tags

df["osmtags"] = ''

for c in df.columns.values:
    if c == 'osmtags':
        continue
    
    df["osmtags"] = np.where(
        df[c] == '' , df.osmtags,
        np.where(df.osmtags == '', df[c], df.osmtags + ';' + df[c])
    )

In [8]:
edf = df[['osmtags']]

obj_to_tbl(edf, otbl)

'/home/jasp/mystuff/multitags_cls.xlsx'